# Sagemaker Jupyter Notebook Integration with Snowflake via local Spark

---
This Notebook shows how to integrate Sagemaker and Snowflake so you can store data in Snowflake and import it into a Jupyter Notebook via the [Spark connector](https://docs.snowflake.net/manuals/user-guide/spark-connector.html). In this particular case, Spark is running on the Notebook server as a local instance. The Jupyter Notebook runs a python kernel . To connect to Snowflake we need to install the Snowflake Connector locally on the Notebook server.

Even though the spark is running on the local Notebook server, it's already much more scalalbe than the Python-connector solution. In this example we will import 5 million rows. However, keep in mind that processing of bigger datasets on a single machine takes its time. Open a termial session from the Jupyter UI in case you like to see what's going on on the Notebook server. 


## Contents

1. [Driver Installation](#Driver-Installation)
1. [Spark Context](#Spark-Context)
1. [Credentials](#Credentials)
1. [Data Import](#Data-Import)

## Driver Installation
The following step installs the latest version of both drivers needed, i.e. JDBC and Spark. If the latest version of the jar files doesn't exist on the Notebook server, all previous jars will be deleted and the newest version will be downloaded from a [maven repository](https://repo1.maven.org/maven2/net/snowflake/). Please note that the the Snowflake driver jars will be installed in their own directory under `/home/ec2-user/snowflake`.

In [1]:
%%bash
SFC_DIR=/home/ec2-user/snowflake
[ ! -d "$SFC_DIR" ] && mkdir $SFC_DIR 
cd $SFC_DIR
PRODUCTS='snowflake-jdbc spark-snowflake_2.11'
for PRODUCT in $PRODUCTS
do
   wget "https://repo1.maven.org/maven2/net/snowflake/$PRODUCT/maven-metadata.xml" 2> /dev/null
   VERSION=$(grep latest maven-metadata.xml | awk -F">" '{ print $2 }' | awk -F"<" '{ print $1 }')
   DRIVER=$PRODUCT-$VERSION.jar
   if [[ ! -e $DRIVER ]]
   then
      rm $PRODUCT* 2>/dev/null
      wget "https://repo1.maven.org/maven2/net/snowflake/$PRODUCT/$VERSION/$DRIVER" 2> /dev/null
   fi
   [ -e maven-metadata.xml ] && rm maven-metadata.xml
done

## Spark Context
The following step configures the spark context. Please note that the inline shell command enumerates the content of the Snowflake jars directory (, which then will be added to the Spark configuration. Please note that you have to stop the spark context in case you want to re-run the notebook. One way to accomplish that is to restart the kernel, or you can just un-comment the line below and execute it.


In [2]:
#sc.stop()

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext,SparkSession
from pyspark.sql.types import *
from sagemaker_pyspark import IAMRole, classpath_jars
from sagemaker_pyspark.algorithms import KMeansSageMakerEstimator

sfc_jars=!ls -d /home/ec2-user/snowflake/*.jar

conf = (SparkConf()
        .set("spark.driver.extraClassPath", (":".join(classpath_jars())+":"+":".join(sfc_jars)))
        .setMaster('local')
        .setAppName('local-spark-test'))
sc=SparkContext(conf=conf)

spark = SQLContext(sc)
sc

<SparkContext master=local appName=local-spark-test>

## Credentials
Credentials can be hard coded but a much more secure way is to stored them in the [Systems Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-paramstore.html). The following step reads the values for the provided keys from the parameter store. These Keys are just an example. You can use the same Keys but you have to create the Key/Value pairs in the parameter store before you can use them here. 

In [4]:
import boto3

params=['/SNOWFLAKE/URL','/SNOWFLAKE/ACCOUNT_ID'
        ,'/SNOWFLAKE/USER_ID','/SNOWFLAKE/PASSWORD'
        ,'/SNOWFLAKE/DATABASE','/SNOWFLAKE/SCHEMA'
        ,'/SNOWFLAKE/WAREHOUSE','/SNOWFLAKE/BUCKET'
        ,'/SNOWFLAKE/PREFIX']

region='us-east-1'

def get_credentials(params):
   ssm = boto3.client('ssm',region)
   response = ssm.get_parameters(
      Names=params,
      WithDecryption=True
   )
   #Build dict of credentials
   param_values={k['Name']:k['Value'] for k in  response['Parameters']}
   return param_values

param_values=get_credentials(params)

## Data Import
The following step reads weather from the [Snowflake Sample Weather Data](https://docs.snowflake.net/manuals/user-guide/sample-data-openweathermap.html) database. Notice, how easy it is to read and transform JSON data. The result set can directly be used to create a pandas data frame. Check out this [JSON tutorial](https://docs.snowflake.net/manuals/user-guide/json-basics-tutorial.html) on the Snowflake documentation site.

Credentials for reading data from Snowflake are passed via an options Array. The specific values have been read in the step above. To ensure that the Notebook server doesn't crash we import only 5 million rows.

In [5]:
sfOptions = {
  "sfURL" : param_values['/SNOWFLAKE/URL'],
  "sfAccount" : param_values['/SNOWFLAKE/ACCOUNT_ID'],
  "sfUser" : param_values['/SNOWFLAKE/USER_ID'],
  "sfPassword" : param_values['/SNOWFLAKE/PASSWORD'],
  "sfDatabase" : param_values['/SNOWFLAKE/DATABASE'],
  "sfSchema" : param_values['/SNOWFLAKE/SCHEMA'],
  "sfWarehouse" : param_values['/SNOWFLAKE/WAREHOUSE'],
}

SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
  .options(**sfOptions) \
  .option("query", \
"select (V:main.temp_max - 273.15) * 1.8000 + 32.00 as temp_max_far, " +\
"       (V:main.temp_min - 273.15) * 1.8000 + 32.00 as temp_min_far, " +\
"       cast(V:time as timestamp) time, " +\
"       V:city.coord.lat lat, " +\
"       V:city.coord.lon lon " +\
"from snowflake_sample_data.weather.weather_14_total limit 5000000").load()
df.describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|      TEMP_MAX_FAR|      TEMP_MIN_FAR|               LAT|               LON|
+-------+------------------+------------------+------------------+------------------+
|  count|           5000000|           5000000|           5000000|           5000000|
|   mean|  70.4678645559177|  68.1955592330811| 28.01917999733069|14.799504624313634|
| stddev|15.307197922767305|15.218515366656854|22.735369707500286|  70.9452732743914|
|    min|          -42.5596|          -42.5596|          -0.03333|          -0.00421|
|    max|             131.9|             131.0|           9.99559|         99.993423|
+-------+------------------+------------------+------------------+------------------+

